<a href="https://colab.research.google.com/github/mansueli/Supa-Migrate/blob/main/Amazon_RDS_to_Supabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Supabase](https://raw.githubusercontent.com/supabase/supabase/master/packages/common/assets/images/supabase-logo-wordmark--light.svg)



#Amazon RDS to Supabase migration guide

In [1]:
#@title Select the `Engine` of the RDS Database & install required resources. { display-mode: "form" }
engine = 'MS SQL' #@param ["Postgres", "MySQL", "MS SQL","SQLite"]

if engine=='Postgres':
  !sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
  !wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
  !sudo apt-get update &>log
  !sudo apt-get -y install postgresql &>log
  !wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/migrate_postgres_project.sh &>log
  !wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/after.sql
  !wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/before.sql
  print("Installed Postgres")
else:
  !wget https://github.com/dimitri/pgloader/archive/refs/tags/v3.6.9.zip &>log
  !apt-get install sbcl unzip libsqlite3-dev make curl gawk freetds-dev libzip-dev &>log
  !unzip v3.6.9.zip -d pgloader &>log
  !make -C ./pgloader/pgloader-3.6.9  &>log
  !./pgloader/pgloader-3.6.9/build/bin/pgloader --help
  print("Pgloader installed to migrated "+engine+ "to Supabase")

./pgloader/pgloader-3.6.9/build/bin/pgloader [ option ... ] command-file ...
./pgloader/pgloader-3.6.9/build/bin/pgloader [ option ... ] SOURCE TARGET
  --help -h                       boolean  Show usage and exit.
  --version -V                    boolean  Displays pgloader version and exit.
  --quiet -q                      boolean  Be quiet
  --verbose -v                    boolean  Be verbose
  --debug -d                      boolean  Display debug level information.
  --client-min-messages           string   Filter logs seen at the console (default: "warning")
  --log-min-messages              string   Filter logs seen in the logfile (default: "notice")
  --summary -S                    string   Filename where to copy the summary
  --root-dir -D                   string   Output root directory. (default: #P"/tmp/pgloader/")
  --upgrade-config -U             boolean  Output the command(s) corresponding to .conf file for
                                           v2.x
  --list-encod

## Set the environment Variables:

In [ ]:
#Source DB variables:
%env HOST=
%env USER=
%env SOURCE_DB=
%env PASSWORD=
%env PORT=
#Supabase variables:
%env SUPABASE_URL=
%env SUPABASE_PASSWORD=

In [1]:
#@title #Running the Migration: { display-mode: "form" }
import os

supabase_url = os.environ['SUPABASE_URL']
if "https://" in supabase_url:
  os.environ["SUPABASE_URL"] = supabase_url.replace('https://','db')

if engine=='MySQL':
  !./pgloader/pgloader-3.6.9/build/bin/pgloader mysql://$USER:$PASSWORD@$HOST:$PORT/$SOURCE_DB pgsql://postgres:$SUPABASE_PASSWORD@$SUPABASE_URL:6543/postgres --before before.sql --after after.sql#&>log
if engine=='MS SQL':
  !./pgloader/pgloader-3.6.9/build/bin/pgloader mssql://$USER:$PASSWORD@$HOST:$PORT/$SOURCE_DB pgsql://postgres:$SUPABASE_PASSWORD@$SUPABASE_URL:6543/postgres --before before.sql --after after.sql #&>log
if engine=='SQLite':
  !./pgloader/pgloader-3.6.9/build/bin/pgloader sqllite://$USER:$PASSWORD@$HOST:$PORT/$SOURCE_DB pgsql://postgres:$SUPABASE_PASSWORD@$SUPABASE_URL:6543/postgres --before before.sql --after after.sql #&>log
if engine=='Postgres':
  sender_db = os.environ['POSTGRES_DATABASE']
  sender_user = os.environ['POSTGRES_USER']
  sender_host = os.environ['POSTGRES_HOST']
  sender_pgpass = os.environ['POSTGRES_PASSWORD']
  filedata = ''
  with open('migrate_postgres_project.sh', 'r') as file :
    filedata = file.read()
    filedata = filedata.replace('POSTGRES_ORIGIN_USERNAME', sender_user)
    filedata = filedata.replace('POSTGRES_ORIGIN_PASSWORD', sender_pgpass.replace("PGPASSWORD=",""))
    filedata = filedata.replace('POSTGRES_ORIGIN_DATABASE', sender_db)
    filedata = filedata.replace('POSTGRES_ORIGIN_HOST', sender_host)
    filedata = filedata.replace('SUPABASE_HOST', os.environ['SUPABASE_HOST'])
    filedata = filedata.replace('SUPABASE_PASSWORD', os.environ['SUPABASE_PASSWORD'])

  with open('migrate_postgres_project.sh', 'w') as file:
    file.write(filedata)

  !bash ./migrate_postgres_project.sh #&>log
print("Migration completed")

KeyError: ignored